In [1]:
from stattlepy import Stattleship
import pandas as pd
from itertools import product

In [3]:
# Transfers Slug from 'Titans vs Cardinals December 10, 2017 at  4:05pm' to 'nfl-2017-2018-ten-ari-2017-12-10-1605'
def func(s):
    """Changes Game Name to Game Slug"""
    l = s.split(' ')
    season_dict = {('September', '2015'): 'nfl-2015-2016', ('October', '2015'): 'nfl-2015-2016', ('November', '2015'): 'nfl-2015-2016', ('December', '2015'): 'nfl-2015-2016', ('January', '2016'): 'nfl-2015-2016', ('February', '2016'): 'nfl-2015-2016', ('September', '2016'): 'nfl-2016-2017', ('October', '2016'): 'nfl-2016-2017', ('November', '2016'): 'nfl-2016-2017', ('December', '2016'): 'nfl-2016-2017', ('January', '2017'): 'nfl-2016-2017', ('February', '2017'): 'nfl-2016-2017', ('September', '2017'): 'nfl-2017-2018', ('October', '2017'): 'nfl-2017-2018', ('November', '2017'): 'nfl-2017-2018', ('December', '2017'): 'nfl-2017-2018', ('January', '2018'): 'nfl-2017-2018', ('February', '2018'): 'nfl-2017-2018'}
    e1 = season_dict[l[3], l[5]]
    team_dict = {'Cardinals': 'ari', 'Falcons': 'atl', 'Ravens': 'bal', 'Bills': 'buf', 'Panthers': 'car', 'Bears': 'chi', 'Bengals': 'cin', 'Browns': 'cle', 'Cowboys': 'dal', 'Broncos': 'den', 'Lions': 'det', 'Packers': 'gb', 'Texans': 'hou', 'Colts': 'ind', 'Jaguars': 'jac', 'Chiefs': 'kc', 'Chargers': 'lac', 'Rams': 'stl', 'Dolphins': 'mia', 'Vikings': 'min', 'Patriots': 'ne', 'Saints': 'no', 'Giants': 'nyg', 'Jets': 'nyj', 'Raiders': 'oak', 'Eagles': 'phi', 'Steelers': 'pit', '49ers': 'sf', 'Seahawks': 'sea', 'Buccaneers': 'tb', 'Titans':'ten', 'Redskins': 'was'}
    e2 = team_dict[l[0]]
    e3 = team_dict[l[2]]
    month_dict = {'September': '09', 'October': '10', 'November': '11', 'December': '12', 'January': '01', 'February': '02'}
    e4 = month_dict[l[3]]
    day_dict = {'1,': '1', '2,': '2', '3,': '3', '4,': '4', '5,': '5', '6,': '6', '7,': '7', '8,': '8', '9,': '9', '10,': '10', '11,': '11', '12,': '12', '13,': '13', '14,': '14', '15,': '15', '16,': '16', '17,': '17', '18,': '18', '19,': '19', '20,': '20', '21,': '21', '22,': '22', '23,': '23', '24,': '24', '25,': '25', '26,': '26', '27,': '27', '28,': '28', '29,': '29', '30,': '30', '31,': '31'}
    e5 = day_dict[l[4]]
    time_dict = {'9:30am': '0930', '12:30pm': '1230', '1:00pm': '1300', '4:05pm': '1605', '4:25pm': '1625', '4:30pm': '1630', '7:10pm': '1910', '8:15pm': '2015', '8:20pm': '2020','8:25pm' : '2025', '8:30pm': '2030', '10:20pm': '2220'}
    e6 = time_dict[l[8]]
    return e1 + '-' + e2 + '-' + e3 + '-' + str(l[5]) + '-' + e4 + '-' + e5 + '-' + e6

In [12]:
# Where it all goes when ready
slugs = []

season_slug = ['nfl-2015-2016', 'nfl-2016-2017', 'nfl-2017-2018']
team_slug = ['nfl-ari', 'nfl-atl', 'nfl-bal', 'nfl-buf', 'nfl-car', 'nfl-chi', 'nfl-cin', 'nfl-cle', 'nfl-dal', 'nfl-den', 'nfl-det', 'nfl-gb', 'nfl-hou', 'nfl-ind', 'nfl-jac', 'nfl-kc', 'nfl-lac', 'nfl-stl', 'nfl-mia', 'nfl-min', 'nfl-ne', 'nfl-no', 'nfl-nyg', 'nfl-nyj', 'nfl-oak', 'nfl-phi', 'nfl-pit', 'nfl-sf', 'nfl-sea', 'nfl-tb', 'nfl-ten', 'nfl-was']

New_query = Stattleship()
Token = New_query.set_token('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

for season, team in product(season_slug, team_slug):
    print(season, team)
    Output = New_query.ss_get_results(sport='football', league='nfl', ep='team_game_logs', season_id=season, team_id=team)
    slugs.append(Output)
    
flattened_list = [item for sub_list in slugs for item in sub_list]

# Loop 0 to 15 for each team
name_list = flattened_list[0]['games'][:]
log = [record['name'] for record in name_list]

# Use to make list of names then put it through function. Will create list of slugs to query to get game_id
games = []
for row in log:
    games.append(row)

df = pd.DataFrame()

# Double up Features, one each for Road and Home Team
# Trying to create columns for winning and losing teams
for name in games:
    name = func(name)
    season = name[:13]
    Output_three = New_query.ss_get_results(sport='football',league='nfl', ep='team_game_logs', season_id=season, game_id=name)
    s = pd.Series()
    s['Name of Winning Team'] = Output_three[0]['team_game_logs'][0]['away_team_outcome'] == 'win' | Output_three[0]['team_game_logs'][1]['home_team_outcome'] == 'win'
    s['Name of Losing Team'] = Output_three[0]['team_game_logs'][0]['away_team_outcome'] == 'loss' | Output_three[0]['team_game_logs'][1]['home_team_outcome'] == 'loss'
    s['Name of Tying Teams'] = Output_three[0]['team_game_logs'][0]['away_team_outcome'] == 'tie' & Output_three[0]['team_game_logs'][1]['home_team_outcome'] == 'tie'
    s['Win'] = Output_three['team_outcome'] == 'win'
    s['Loss'] = Output_three['team_outcome'] == 'loss'
    s['Tie'] = Output_three ['team_outcome'] == 'tie'
    s['Wins'] = Output_three['games_won']
    s['Losses'] = Output_three['games_lost']
    s['Ties'] = Output_three['games_tied']
    s['Points'] = Output_three['points']
    s['Rushing Atts'] = Output_three['rushing_plays']
    s['Rushing Yds'] = Output_three['rushing_gross_yards']
    s['Rushing TDs'] = Output_three['rushing_touchdowns']
    s['Rushing First Downs'] = Output_three['rushing_first_downs']
    s['Passing Atts'] = Output_three['passing_plays_attempted']
    s['Passing Completions'] = Output_three['passing_plays_completed']
    s['Passing Yds'] = Output_three['passing_gross_yards']
    s['Passing TDs'] = Output_three['passing_touchdowns']
    s['Passing First Downs'] = Output_three['passing_first_downs']
    s['Total Yds'] = Output_three['rushing_gross_yards'] + Output_three['passing_gross_yards']
    s['Total TDs'] = Output_three['total_touchdowns']
    s['First Downs'] = Output_three['total_first_downs']
    s['Third Down Attempts'] = Output_three['third_downs_attempted']
    s['Third Down Successes'] = Output_three['third_downs_succeeded']
    s['Fourth Down Attempts'] = Output_three['fourth_downs_attempted']
    s['Fourth Down Successes'] = Output_three['fourth_downs_succeeded']
    s['Red Zone Attempts'] = Output_three['red_zone_scores_attempted']
    s['Red Zone Successes'] = Output_three['red_zone_scores_succeeded']
    s['Goal To Go Attempts'] = Output_three['goal_to_go_attempted']
    s['Goal To Go Successes'] = Output_three['goal_to_go_succeeded']
    s['Two Points Conversion Attempts'] = Output_three['extra_point_passes_attempted'] + Output_three['extra_point_rushes_attempted']
    s['Two Points Conversion Successes'] = Output_three['extra_point_passes_succeeded'] + Output_three['extra_point_rushes_succeeded']
    s['Time of Possession'] = Output_three['time_of_possession_secs']
    s['Sacks Given Up'] = Output_three['passing_plays_sacked']
    s['Sacks Yds Given Up'] = Output_three['passing_sacked_yards']
    s['INTs'] = Output_three['interception_returns']
    s['INTs Given Up'] = Output_three['passing_plays_intercepted']
    s['INT TDs'] = Output_three['interception_touchdowns']
    s['Fumbles'] = Output_three['fumbles']
    s['Fumbles Lost'] = Output_three['fumbles_lost']
    s['Fumbles Recovered'] = Output_three['fumbles_recovered_defense']
    s['Fumble TDs'] = Output_three['fumble_touchdowns']
    s['Safeties'] = Output_three['safeties']
    s['Field Goal Attempts'] = Output_three['field_goals_attempted']
    s['Made Field Goals'] = Output_three['field_goals_succeeded']
    s['Kickoffs'] = Output_three['kickoffs']
    s['Kickoff Returns'] = Output_three['kickoff_returns']
    s['Kickoff Return YDs'] = Output_three['kickoff_return_yards']
    s['Kickoff Return TDs'] = Output_three['kickoff_return_touchdowns']
    s['Touchbacks'] = Output_three['kickoff_touchbacks']
    s['Punt Returns'] = Output_three['punt_returns']
    s['Punt Return Yds'] = Output_three['punt_return_yards']
    s['Punt Return TDs'] = Output_three['punt_return_touchdowns']
    s['Punts'] = Output_three['punting_plays']
    s['Punting Net Yards'] = Output_three['punting_net_yards_avg']
    s['Penalties'] = Output_three['penalties']
    s['Penalty Yards'] = Output_three['penalty_yards']
    s['Penalty First Downs'] = Output_three['penalty_first_downs']
    df = pd.concat(df, s.to_frame())
# Picking out different values for output

# Loop through every game, 50 features to start. Figure out unique fields to keep
df.head()

Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2015-2016&season_id=nfl-2016-2017&season_id=nfl-2017-2018&team_id=nfl-ari&team_id=nfl-atl&team_id=nfl-bal&team_id=nfl-buf&team_id=nfl-car&team_id=nfl-chi&team_id=nfl-cin&team_id=nfl-cle&team_id=nfl-dal&team_id=nfl-den&team_id=nfl-det&team_id=nfl-gb&team_id=nfl-hou&team_id=nfl-ind&team_id=nfl-jac&team_id=nfl-kc&team_id=nfl-lac&team_id=nfl-stl&team_id=nfl-mia&team_id=nfl-min&team_id=nfl-ne&team_id=nfl-no&team_id=nfl-nyg&team_id=nfl-nyj&team_id=nfl-oak&team_id=nfl-phi&team_id=nfl-pit&team_id=nfl-sf&team_id=nfl-sea&team_id=nfl-tb&team_id=nfl-ten&team_id=nfl-was&page=1
Stattleship API request complete
Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2015-2016&season_id=nfl-2016-2017&season_id=nfl-2017-2018&team_id=nfl-ari&team_id=nfl-atl&team_id=nfl-bal&team_id=nfl-buf&team_id=nfl-car&team_id=nfl-chi&team_id=nfl-ci

TypeError: list indices must be integers, not str

In [28]:
New_query = Stattleship()
Token = New_query.set_token('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

games = ['Titans vs Cardinals December 10, 2017 at  4:05pm']

df = pd.DataFrame()

# **Issue with Slicing**
for name in games:
    name = func(name)
    season = name[:13]
    Output_three = New_query.ss_get_results(sport='football',league='nfl', ep='team_game_logs', season_id=season, game_id=name)

if Output_three[0]['team_game_logs'][0]['away_team_outcome'] == 'win':
    print(Output_three[0]['away_teams'][0]['nickname'])

# For Road Teams: Output_three[0]['team_game_logs'][0]['away_team_outcome']
# For Home Teams: Output_three[0]['team_game_logs'][1]['home_team_outcome']
# 0 at end for Road Team, 1 at end for Home Team
# Each has 'is_away_team' and 'is_home_team' that equal True or False
# Find name of Road Team: Output_three[0]['away_teams'][0]['nickname']
# Find name for Home Team: Output_three[0]['home_teams'][0]['nickname']

# If Square, find index I want
# If Curly, find name of field

Making Initial API Request
<Response [200]>
https://api.stattleship.com/football/nfl/team_game_logs?season_id=nfl-2017-2018&game_id=nfl-2017-2018-ten-ari-2017-12-10-1605&page=1
Stattleship API request complete


In [ ]:
# Querying by Team by Season
Output = New_query.ss_get_results(sport='football',league='nfl', ep='team_game_logs', season_id='nfl-2017-2018')
Output

# Querying by Team by Game (only shows one team's stats)
Output_two = New_query.ss_get_results(sport='football',league='nfl', ep='team_game_logs', season_id='nfl-2017-2018',team_id='nyg')
Output_two

# Querying by Team by Game (shows both teams' stats)
Output_three = New_query.ss_get_results(sport='football',league='nfl', ep='team_game_logs', season_id='nfl-2017-2018', game_id='nfl-2017-2018-ari-det-2017-09-10-1300')
Output_three